# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
!pip install pandas_profiling
import pandas as pd # библиотека Pandas
import pandas_profiling # библиотека Pandas_Profiling
from sklearn.linear_model import LinearRegression # модель линейной регрессии
from sklearn.preprocessing import StandardScaler # модуль для стандартизации признаков
from sklearn.model_selection import train_test_split # модуль для разбиения на обучающую и валидационную выборки
from sklearn.metrics import mean_squared_error # среднеквадратичное отклонение
from sklearn.preprocessing import PolynomialFeatures # полином для признаков
import numpy as np # библиотека NumPy

/tmp/ipykernel_461/487621852.py:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling # библиотека Pandas_Profiling


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Данные содержат следующую информацию:
- `id` — уникальный идентификатор скважины;
- `f0`, `f1`, `f2` — три признака точек;
- `product` — объём запасов в скважине (тыс. баррелей).

Проверим, датафрейм регоина №0.

In [4]:
pandas_profiling.ProfileReport(data_0)

Из важного можно заметить, что 10 полей `id` не являются уникальными, дубликаты нужно удалить.

In [5]:
data_0.drop_duplicates(subset='id', inplace=True)
data_0.reset_index(inplace=True)

Проверим регион №1.

In [6]:
pandas_profiling.ProfileReport(data_1)

В данном датасете также можно увидеть неуникальные значения `id`, кроме того, 8.3% данных содержат нули в значении `product`. Возможно это ошибка, во всяком случае заполнить их правильно не получится, придется нулевые значения удалить.

In [7]:
data_1.drop_duplicates(subset='id', inplace=True)
data_1 = data_1.query('product != 0').reset_index()

Проверим регион №2.

In [8]:
pandas_profiling.ProfileReport(data_2)

Здесь тоже есть неуникальные значения `id`, удалим их.

In [9]:
data_2.drop_duplicates(subset='id', inplace=True)
data_2.reset_index(inplace=True)

## Обучение и проверка модели

Для каждого региона следует обучить отдельные модели, т.к. признаки, влияющие на запас сырья могут отличаться своим вкладом в целевой признак. Начнем с нулевого региона.

### Регион `№0`

Проверим корреляцию признаков.

In [10]:
data_0.corr()

,index,f0,f1,f2,product
index,1.000000,0.003416,0.006862,0.000071,-0.000604
f0,0.003416,1.000000,-0.440717,-0.003211,0.143504
f1,0.006862,-0.440717,1.000000,0.001764,-0.192351
f2,0.000071,-0.003211,0.001764,1.000000,0.483631
product,-0.000604,0.143504,-0.192351,0.483631,1.000000


Наибольший вклад вносит признак `f2`, но остальные тоже оказывают сильное влияние, обучим модель на основе всех признаков (кроме `id`).

In [11]:
features_0 = data_0.drop(columns=['product', 'id'])
target_0 = data_0['product']
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=75)

Стандартизируем данные.

In [12]:
scaler_0 = StandardScaler()
scaler_0.fit(features_0)
X_train_0 = scaler_0.transform(X_train_0)
X_test_0 = scaler_0.transform(X_test_0)

In [13]:
model_0 = LinearRegression()
model_0.fit(X_train_0, y_train_0)
pred_0 = model_0.predict(X_test_0)
print('Метрика RMSE:', mean_squared_error(y_test_0, pred_0)**0.5)

Метрика RMSE: 37.85469032271623


Получилось довольно большое отклонение, сравним модель с константной.

In [14]:
mean_0 = pd.Series(y_test_0.mean(), index=y_test_0.index)
print('Метрика RMSE для константной модели:', mean_squared_error(y_test_0, mean_0)**0.5)

Метрика RMSE для константной модели: 44.57213557109568


Константная модель показывает результат хуже на 10 пунктов. Попробуем преобразовать признаки в полином, чтобы улучшить метрику.

In [15]:
poly_0 = PolynomialFeatures(4)
poly_X_train_0 = poly_0.fit_transform(X_train_0)
poly_X_test_0 = poly_0.fit_transform(X_test_0)
model_0.fit(poly_X_train_0, y_train_0)
poly_pred_0 = model_0.predict(poly_X_test_0)
print('Метрика RMSE:', mean_squared_error(y_test_0, poly_pred_0)**0.5)

Метрика RMSE: 37.449410252937795


Особой разницы нет, стало лучше на 0.4 пункта.

Оценим средний запас сырья по предсказаниям модели.

In [16]:
poly_pred_0.mean()

92.2163652804685

**Средний запас сырья в скважинах региона `№0` составил 92 тысячи баррелей. Среднеквадратичное отклонение — 37.33.**

### Регион `№1`

Проверим корреляцию признаков для региона №1.

In [17]:
data_1.corr()

,index,f0,f1,f2,product
index,1.000000,-0.000657,0.002647,0.001552,0.001398
f0,-0.000657,1.000000,0.181424,0.126414,0.096441
f1,0.002647,0.181424,1.000000,0.031023,0.023144
f2,0.001552,0.126414,0.031023,1.000000,0.999329
product,0.001398,0.096441,0.023144,0.999329,1.000000


Больше всех вклад вносит признак `f2`. Попробуем обучить две модели: одна учитывает все признаки, вторая только `f2`.

In [18]:
features_1 = data_1.drop(columns=['product', 'id'])
target_1 = data_1['product']
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=75)
scaler_1 = StandardScaler()
scaler_1.fit(features_1)
X_train_1 = scaler_1.transform(X_train_1)
X_test_1 = scaler_1.transform(X_test_1)
model_1 = LinearRegression()
model_1.fit(X_train_1, y_train_1)
pred_1 = model_1.predict(X_test_1)
print('Метрика RMSE:', mean_squared_error(y_test_1, pred_1)**0.5)

Метрика RMSE: 0.8866615961747084


Среднеквадратичное отклонение получилось значительно меньше, чем в `нулевом` регионе, попробуем обучить лишь на признаке `f2`.

In [19]:
features_1_f2 = data_1.drop(columns=['product', 'id', 'f0', 'f1'])
target_1_f2 = data_1['product']
X_train_1_f2, X_test_1_f2, y_train_1_f2, y_test_1_f2 = train_test_split(
    features_1_f2, target_1_f2, test_size=0.25, random_state=75)
model_1_f2 = LinearRegression()
model_1_f2.fit(X_train_1_f2, y_train_1_f2)
pred_1_f2 = model_1_f2.predict(X_test_1_f2)
print('Метрика RMSE:', mean_squared_error(y_test_1_f2, pred_1_f2)**0.5)

Метрика RMSE: 1.5660815297026807


Метрика стала хуже, оставим модель, в которой обучение происходило на всех признаках. Проверим константную модель.

In [20]:
mean_1 = pd.Series(y_test_1.mean(), index=y_test_1.index)
print('Метрика RMSE для константной модели:', mean_squared_error(y_test_1, mean_1)**0.5)

Метрика RMSE для константной модели: 43.08416457302911


Константная модель показала себя значительно хуже, чем обученная.

Оценим средний запас сырья по прогнозам модели.

In [21]:
pred_1.mean()

75.00415456765336

**Средний запас сырья в скважинах региона `№1` — 75 тысяч баррелей. Среднеквадратичное отклонение составило — 0.88.**

### Регион `№2`

Проверим корреляцию признаков для региона №2.

In [22]:
data_2.corr()

,index,f0,f1,f2,product
index,1.000000,-0.000887,0.001683,0.005369,0.000659
f0,-0.000887,1.000000,0.000506,-0.000452,-0.001978
f1,0.001683,0.000506,1.000000,0.000753,-0.001055
f2,0.005369,-0.000452,0.000753,1.000000,0.445867
product,0.000659,-0.001978,-0.001055,0.445867,1.000000


Первые два признака почти не влияют на количество запасов в скважине, попробуем обучить модели так же, как в регионе №1.

In [23]:
features_2 = data_2.drop(columns=['product', 'id'])
target_2 = data_2['product']
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=75)
scaler_2 = StandardScaler()
scaler_2.fit(features_2)
X_train_2 = scaler_2.transform(X_train_2)
X_test_2 = scaler_2.transform(X_test_2)
model_2 = LinearRegression()
model_2.fit(X_train_2, y_train_2)
pred_2 = model_2.predict(X_test_2)
print('Метрика RMSE:', mean_squared_error(y_test_2, pred_2)**0.5)

Метрика RMSE: 40.08631854496551


Метрика получилась не самая лучшая, проверим модель на признаке `f2`.

In [24]:
features_2_f2 = data_2.drop(columns=['product', 'id', 'f0', 'f1'])
target_2_f2 = data_2['product']
X_train_2_f2, X_test_2_f2, y_train_2_f2, y_test_2_f2 = train_test_split(
    features_2_f2, target_2_f2, test_size=0.25, random_state=75)
model_2_f2 = LinearRegression()
model_2_f2.fit(X_train_2_f2, y_train_2_f2)
pred_2_f2 = model_2_f2.predict(X_test_2_f2)
print('Метрика RMSE:', mean_squared_error(y_test_2_f2, pred_2_f2)**0.5)

Метрика RMSE: 40.08579601342574


Едва заметно, но метрика улучшилась, попробуем добавить полином к признакам.

In [25]:
poly_2 = PolynomialFeatures(4)
poly_X_train_2 = poly_2.fit_transform(X_train_2)
poly_X_test_2 = poly_2.fit_transform(X_test_2)
model_2.fit(poly_X_train_2, y_train_2)
poly_pred_2 = model_2.predict(poly_X_test_2)
print('Метрика RMSE:', mean_squared_error(y_test_2, poly_pred_2)**0.5)

Метрика RMSE: 38.13197690407122


Метрику удалось улучшить на два пункта. Оценим средний запас в скважинах региона №2.

In [26]:
poly_pred_2.mean()

95.14940211755037

**Средний запас в скважинах региона `№2` — 95 тысяч баррелей. Среднеквадратичное отклонение — 38.12.**

### Вывод по обучению и проверки моделей.

Модель обучилась и спрогнозировала следующие данные:
 - регион №0: средний запас сырья в скважинах — 92 тысячи баррелей, среднеквадратичное отклонение модели — 37.33;
 - регион №1: средний запас сырья в скважинах — 75 тысяч баррелей, среднеквадратичное отклонение модели — 0.88;
 - регион №2: средний запас сырья в скважинах — 95 тысяч баррелей, среднеквадратичное отклонение модели — 38.12.

Модель в регионе №1 оказалась самой точной, при этом среднее количество запасов сырья в нем значительно меньше, чем в двух других.

Модель в регионе №2 показала, что в нем находятся самые богатые скважины.

## Подготовка к расчёту прибыли

Известно, что будут разрабатываться 200 месторождений, бюджет на разработку — 10 млрд рублей. Доход с 1 тысячи баррелей — 450 тысяч рублей. Формула для расчёта безубыточного запаса сырья:

$$V = \frac{10 млрд}{450 тысяч * 200 месторождений}$$

In [27]:
N = 200
S = 10_000_000_000
Y = 450_000
V = S/(N*Y)
print(V)

111.11111111111111


Средний запас сырья в разрабатываемых скважинах должен составлять не менее 111.1 тысяч баррелей. А всего необходимое количество баррелей для добычи должно быть равно:

In [28]:
print(V*N)

22222.222222222223


Чтобы избежать убытка, необходимо добыть 22222 тысяч баррелей сырья.

По всем регионам нет таких средних показателей, поэтому понадобится искать самые богатые скважины.

## Расчёт прибыли и рисков 

### Функция для расчета прибыли

Напишем функцию, которая расчитывает прибыль с месторождений региона.

In [29]:
def revenue(data):
    return int(data['product'].sum() * Y - S)

Итак, возьмем предсказания моделей на валидационной выборке для лучших скважин и найдем прибыль по реальным показателям.

### Регион `№0`

Создадим датасет на основе валидационного вместе с предсказаниями.

In [30]:
pd.options.mode.chained_assignment = None # убираем предупреждения

In [52]:
region_0 = data_0.iloc[y_test_0.index]
region_0['pred'] = poly_pred_0

Найдем 200 лучших скважин по мнению модели и найдем с них прибыль.

In [48]:
best_region_0 = region_0.sort_values(by='pred', ascending=False).head(200)
print(f"Прибыль с региона составит: {revenue(best_region_0):,} рублей.")

Прибыль с региона составит: 3,313,528,197 рублей.


По мнению модели, регион №0 принесет почти 3.282 млрд прибыли с лучших скважин.

### Регион `№1`

Повторяем то же, что и в предыдущем регионе.

In [33]:
region_1 = data_1.iloc[y_test_1.index]
region_1['pred'] = pred_1
best_region_1 = region_1.sort_values(by='pred', ascending=False).head(200)
print(f"Прибыль с региона составит: {revenue(best_region_1):,} рублей.")

Прибыль с региона составит: 2,415,086,696 рублей.


С региона №1 прибыль составила почти на 1 млрд меньше, чем в регионе №0 — 2.415 млрд.

### Регион `№2`

In [34]:
region_2 = data_2.iloc[y_test_2.index]
region_2['pred'] = poly_pred_2
best_region_2 = region_2.sort_values(by='pred', ascending=False).head(200)
print(f"Прибыль с региона составит: {revenue(best_region_2):,} рублей.")

Прибыль с региона составит: 3,489,822,476 рублей.


Регион №2 показал результат 3.489 млрд рублей прибыли.

### Вывод по расчету прибыли с лучших скважин

Прибыль по регионам получилась следующая:

 - регион №0: 3.282 млрд рублей;
 - регион №1: 2.415 млрд рублей;
 - регион №2: 3.489 млрд рублей.

Наибольшую возможную прибыль модель прогнозирует в регионе №2, при выборе лучших скважин из всех имеющихся.

## Оценка рисков методом Bootstrap

Известно, что будет исследовано 500 скважин, из которых 200 самых лучших будут разрабатываться. Мы применим технику Bootstrap, чтобы найти наименее рискованный регион с точки зрения статистики. Количество выборок для Bootstrap будет равно 1000.

Напишем функцию для оценки рисков.

In [49]:
def bootstrap(data):
    state = np.random.RandomState(75)
    values = [] # список для прибыли с каждой выборки
    quantile_values = [] # список для поиска доверительного интервала
    for i in range(1000):
        data_subsample = data.sample(random_state=state, n=500, replace=True) # создаем случайную выборку из 500 скважин
        data_subsample = data_subsample.sort_values(by='pred', ascending=False).head(200) # оставляем из них 200 лучших
        values.append(revenue(data_subsample)) # считаем прибыль с 200 лучших скважин и добавляем в список
    values = pd.Series(values)
    lower = values.quantile(0.025) # ищем 0.025 квантиль
    upper = values.quantile(0.975) # ищем 0.975 квантиль
    return int(lower), int(upper), int(values.mean()), values[values < 0].count() / 1000 * 100

### Проверка рисков

Проверим риски для всех регионов.

In [53]:
region_0_lower_quantile, region_0_upper_quantile,  region_0_mean_revenue, region_0_risk = bootstrap(region_0)
region_1_lower_quantile, region_1_upper_quantile, region_1_mean_revenue, region_1_risk = bootstrap(region_1)
region_2_lower_quantile, region_2_upper_quantile, region_2_mean_revenue, region_2_risk = bootstrap(region_2)

print(f'Доверительный интервал 95% лежит между: {region_0_lower_quantile:,} и {region_0_upper_quantile:,} рублей, а средняя прибыль составит {region_0_mean_revenue:,} рублей')
print(f'Доверительный интервал 95% лежит между: {region_1_lower_quantile:,} и {region_1_upper_quantile:,} рублей, а средняя прибыль составит {region_1_mean_revenue:,} рублей')
print(f'Доверительный интервал 95% лежит между: {region_2_lower_quantile:,} и {region_2_upper_quantile:,} рублей, а средняя прибыль составит {region_2_mean_revenue:,} рублей')

Доверительный интервал 95% лежит между: 7,137,331 и 1,017,768,564 рублей, а средняя прибыль составит 500,225,424 рублей
Доверительный интервал 95% лежит между: 347,750,841 и 1,126,261,057 рублей, а средняя прибыль составит 740,589,064 рублей
Доверительный интервал 95% лежит между: 67,201,301 и 1,108,986,438 рублей, а средняя прибыль составит 573,492,142 рублей


Проверим риски:

In [51]:
print('Риск убытков в регионе №0:', region_0_risk, '%')
print('Риск убытков в регионе №1:', region_1_risk, '%')
print('Риск убытков в регионе №2:', region_2_risk, '%')

Риск убытков в регионе №0: 4.6 %
Риск убытков в регионе №1: 0.0 %
Риск убытков в регионе №2: 1.5 %


Итак, лучший результат по прибыльности показал регион №1, где с вероятностью 95% прибыль составит от 347 млн до 1.1 млрд рублей, а средняя прибыль 740.589 миллионов рублей, а вероятность убытков 0%. То есть безпроигрышный вариант, в отличие от региона №0 с вероятностью убытков 2.4%, и региона №2 с вероятностью 1.5%.

## Вывод

В данной работе предлагалось три набора данных о нефтяных скважинах в трех разных регионах. Данные содержали уникальный идентификатор для каждой скважины, три признака, от которых зависил объем сырья, а также истинное количество сырья.

В работе требовалось обучить модель линейной регрессии для каждого региона для определения запасов сырья в скважине. Признаки были проверены на корреляцию, в некоторых моделях признаки приводились к виду полинома. Основная метрика для оценки точности модели была выбрана RMSE, также модель сравнивалась с константной.

После обучения моделей, требовалось найти наиболее богатые скважины, оценить прибыльность добычи сырья с учетом бюджета на разработку и стоимости 1 барреля нефти. Регион №2 оказался самых перспективным для добычи, где лучшие скважины по оценкам модели способны принести доход в размере 3.5 млрд рублей.

Также были оценены риски при помощи метода `Bootstrap`, при условии, что оцениваются лишь 500 скважин и выбирают 200 лучших из них. Здесь лидером оказался регион №1, где разработка скважин с вероятностью 95% принесет прибыль от 347 млн до 1.1 млрд рублей, а средняя прибыль составит 740 млн рублей, без риска получить убыток.